## [Interactive Technical Analysis Tool in Python (SMA, RSI, BBANDS)](https://www.youtube.com/watch?v=3HP-YbFuL5U)
### NeuralNine

In [1]:
# !pip install plotly
# !pip install nbformat
# !pip install nbformat>=4.2.0


In [2]:
import nbformat
print(nbformat.__version__)


5.10.4


In [3]:
# get data
import pandas as pd
from sqlalchemy import create_engine, inspect

# Create an SQLite engine
engine = create_engine('sqlite:///data/ESNQ_DB.db')

table_names = inspect(engine).get_table_names()
print(table_names)
ticker = 'NQ1D'
df = pd.read_sql(ticker, engine)
df = df[-500:]
print(df.info())

['ES1D', 'ES1M', 'ES30M', 'ES4H', 'ES5M', 'NQ1D', 'NQ1M', 'NQ30M', 'NQ4H', 'NQ5M']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 5878 to 6377
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     500 non-null    int64  
 1   datetime  500 non-null    object 
 2   symbol    500 non-null    object 
 3   open      500 non-null    float64
 4   high      500 non-null    float64
 5   low       500 non-null    float64
 6   close     500 non-null    float64
 7   volume    500 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 31.4+ KB
None


In [4]:
# pre option on df
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.drop(['index','symbol','volume'], axis=1, inplace=True)
df.tail()

,open,high,low,close
datetime,,,,
2024-09-08 17:00:00,18400.00,18734.50,18386.00,18692.25
2024-09-09 17:00:00,18727.00,18878.50,18582.25,18864.00
2024-09-10 17:00:00,18837.75,19293.50,18547.00,19271.00
2024-09-11 17:00:00,19241.50,19483.50,19185.00,19447.25
2024-09-12 17:00:00,19419.50,19577.25,19398.25,19530.25


In [5]:
# use pandas_ta build indicators
import pandas_ta as ta
df['sma'] = ta.sma(df['close'], length=20)
df['rsi'] = ta.rsi(df['close'], length=14)
df.ta.bbands(length=20, std=2, append=True)

# df['macd'], df['macd_signal'], df['macd_hist'] = ta.macd(df['close'], fast=12, slow=26, signal=9)
df

,open,high,low,close,sma,rsi,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
datetime,,,,,,,,,,,
2022-09-19 17:00:00,12045.75,12092.50,11828.25,11922.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-20 17:00:00,11946.00,12140.50,11647.50,11710.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-21 17:00:00,11661.25,11796.00,11510.00,11565.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-22 17:00:00,11583.75,11600.25,11229.00,11376.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-25 17:00:00,11362.50,11537.75,11256.50,11316.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-08 17:00:00,18400.00,18734.50,18386.00,18692.25,19337.5750,40.896020,18484.211462,19337.5750,20190.938538,8.825962,0.121893
2024-09-09 17:00:00,18727.00,18878.50,18582.25,18864.00,19348.8000,44.127719,18527.144208,19348.8000,20170.455792,8.493093,0.204986
2024-09-10 17:00:00,18837.75,19293.50,18547.00,19271.00,19357.1375,50.969413,18542.220442,19357.1375,20172.054558,8.419810,0.447150


In [6]:
# build visualization
import plotly.io as pio
from plotly.subplots import make_subplots
# pio.templates.default = "plotly_white"
import plotly.graph_objects as go
# pio.renderers.default='browser'

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    vertical_spacing=0.2, row_heights=[0.7, 0.3],
                    subplot_titles=[f'{ticker} Price and Indicators','RSI'])

cadlestick = go.Candlestick(
    x=df.index,
    open=df.open,
    high=df.high,
    low=df.low,
    close=df.close,
    name='Price'
)

sma_line = go.Scatter(
    x = df.index,
    y = df.sma,
    line={'color':'orange', 'width':2},
    name='SMA'
)

upper_bb = go.Scatter(
    x=df.index,
    y=df['BBU_20_2.0'],
    line={'color':'red', 'width':2},
    name='Upper BB'
)

lower_bb = go.Scatter(
    x=df.index,
    y=df['BBL_20_2.0'],
    line={'color':'red', 'width':2},
    name='Lower BB'
)

middle_bb = go.Scatter(
    x=df.index,
    y=df['BBM_20_2.0'],
    line={'color':'blue', 'width':2},
    name='Middle BB'
)

fig.add_trace(cadlestick, row=1, col=1)
fig.add_trace(sma_line, row=1, col=1)
fig.add_trace(upper_bb, row=1, col=1)
fig.add_trace(lower_bb, row=1, col=1)
fig.add_trace(middle_bb, row=1, col=1)

rsi = go.Scatter(
    x=df.index,
    y=df.rsi,
    line={'color':'purple', 'width':2},
    name='RSI'
)

fig.add_trace(rsi, row=2, col=1)

mark30 = go.layout.Shape(
    type='line',
    x0=df.index[0],
    y0=30,
    x1=df.index[-1],
    y1=30,
    line=dict(
        color='gray',
        width=1,
        dash='dash'
    )
)

mark70 = go.layout.Shape(
    type='line',
    x0=df.index[0],
    y0=70,
    x1=df.index[-1],
    y1=70,
    line={'color':'gray', 'width':1, 'dash':'dash'}
)

fig.add_shape(mark30, row=2, col=1)
fig.add_shape(mark70, row=2, col=1)

fig.update_yaxes(range=[0,100], row=2, col=1)

fig.update_layout(
    title=f'{ticker} Technical Analyst',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    height=800,
    template='plotly_white') # 'plotly_dark'

fig.show()